In [87]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import math
import os
plotly.offline.init_notebook_mode(connected=True)
os.chdir(r'E:\桌面\建投工作文件\库存周期')
os.listdir()

['CEIC宏观数据.xlsx',
 'CEIC数据库情况.md',
 'temp-plot.html',
 '~$CEIC宏观数据.xlsx',
 '~$细分行业月度收盘价.xlsx',
 '上市公司分类指引（2012）.doc',
 '各行业收益率.xlsx',
 '国民经济行业分类.pdf',
 '宏观数据来源.pdf',
 '细分行业月度收盘价.xlsx',
 '行业分类(1).xlsx']

# 准备数据

In [186]:
def economic_data(source, sheet_name):
    data = pd.read_excel(source, sheet_name=sheet_name, header=0, index_col=0,
                         parse_dates=True, engine='openpyxl').fillna(method='ffill').dropna(how='all')
    data = data.applymap(lambda x: math.copysign(200, x) if abs(x) > 200 else round(x, 1))  # 绝对值超过200的，限制到200
    return data

def stock_data(source, sheet_name):
    data = pd.read_excel(source, sheet_name=sheet_name, header=0, index_col=0,
                         parse_dates=True, engine='openpyxl').pct_change().fillna(method='ffill').dropna(how='all')
    data = data.applymap(lambda x: round(x * 100, 1))  # 进行量纲转化
    data.index = data.index.map(lambda x: x+pd.Timedelta(days=1))
    return data

In [117]:
ppi = economic_data('CEIC宏观数据.xlsx', 'PPI')
ppi.columns

Index(['工业生产者出厂价格指数', '生产资料', '生产资料:采掘工业', '生产资料:原材料工业', '生产资料:加工工业', '生活资料',
       '生活资料:食品类', '生活资料:衣着类', '生活资料:一般日用品', '生活资料:耐用消费品', '煤炭开采和洗选业',
       '石油和天然气开采业', '黑色金属矿采选业', '有色金属矿采选业', '非金属矿采选业', '开采专业及辅助性活动', '其他采矿业',
       '农副食品加工业', '食品制造业', '酒、饮料和精制茶制造业', '烟草制品业', '纺织业', '纺织服装、服饰业',
       '皮革、毛皮、羽毛及其制品和制鞋业', '木材加工及木、竹、藤、棕、草制品业', '家具制造业', '造纸和纸制品业',
       '印刷和记录媒介复制业', '文教、工美、体育和娱乐用品制造业', '石油、煤炭及其他燃料加工业', '化学原料和化学制品制造业',
       '医药制造业', '化学纤维制造业', '非金属矿物制品业', '黑色金属冶炼和压延加工业', '有色金属冶炼和压延加工业', '金属制品业',
       '通用设备制造业', '专用设备制造业', '汽车制造业', '铁路、船舶、航空航天和其他运输设备制造业', '电气机械和器材制造业',
       '计算机、通信和其他电子设备制造业', '仪器仪表制造业', '其他制造业', '废弃资源综合利用业', '金属制品、机械和设备修理业',
       '电力、热力生产和供应业', '燃气生产和供应业', '水的生产和供应业'],
      dtype='object')

In [143]:
income = economic_data('CEIC宏观数据.xlsx', '营业收入')
income.columns

Index(['煤炭开采和洗选业', '石油和天然气开采业', '黑色金属矿采选业', '有色金属矿采选业', '非金属矿采选业',
       '开采专业和辅助性活动', '其他采矿业', '农副食品加工业', '食品制造业', '酒、饮料和精制茶制造业', '烟草制品业',
       '纺织业', '纺织服装、服饰业', '皮革、毛皮、羽毛和其制品和制鞋业', '木材加工和木、竹、藤、棕、草制品业', '家具制造业',
       '造纸和纸制品业', '印刷和记录媒介复制业', '文教、工美、体育和娱乐用品制造业', '石油、煤炭和其他燃料加工业',
       '化学原料和化学制品制造业', '医药制造业', '化学纤维制造业', '非金属矿物制品业', '黑色金属冶炼和压延加工业',
       '有色金属冶炼和压延加工业', '金属制品业', '通用设备制造业', '专用设备制造业', '汽车制造业',
       '铁路、船舶、航空航天和其他运输设备制造业', '电气机械和器材制造业', '计算机、通信和其他电子设备制造业', '仪器仪表制造业',
       '其他制造业', '废弃资源综合利用业', '金属制品、机械和设备修理业', '电力、热力生产和供应业', '燃气生产和供应业',
       '水的生产和供应业'],
      dtype='object')

In [176]:
stock = economic_data('CEIC宏观数据.xlsx', '产成品库存')
stock.index = stock.index.map(lambda x: pd.Timestamp(x)+pd.Timedelta(days=1))
stock.columns

Index(['煤炭开采和洗选业', '石油和天然气开采业', '黑色金属矿采选业', '有色金属矿采选业', '非金属矿采选业',
       '开采专业和辅助性活动', '其他采矿业', '农副食品加工业', '食品制造业', '酒、饮料和精制茶制造业', '烟草制品业',
       '纺织业', '纺织服装、服饰业', '皮革、毛皮、羽毛和其制品和制鞋业', '木材加工和木、竹、藤、棕、草制品业', '家具制造业',
       '造纸和纸制品业', '印刷和记录媒介复制业', '文教、工美、体育和娱乐用品制造业', '石油、煤炭和其他燃料加工业',
       '化学原料和化学制品制造业', '医药制造业', '化学纤维制造业', '非金属矿物制品业', '黑色金属冶炼和压延加工业',
       '有色金属冶炼和压延加工业', '金属制品业', '通用设备制造业', '专用设备制造业', '汽车制造业',
       '铁路、船舶、航空航天和其他运输设备制造业', '电气机械和器材制造业', '计算机、通信和其他电子设备制造业', '仪器仪表制造业',
       '其他制造业', '废弃资源综合利用业', '金属制品、机械和设备修理业', '电力、热力生产和供应业', '燃气生产和供应业',
       '水的生产和供应业'],
      dtype='object')

In [180]:
inc_stk = (income - stock).dropna(how='all')

In [187]:
statistic_ind = stock_data('细分行业月度收盘价.xlsx', '证监会二级')
statistic_ind.columns

Index(['农副食品加工', '食品制造', '饮料和精制茶制造', '纺织', '纺织服装服饰', '皮革制鞋', '木材加工', '家具制造',
       '造纸和纸制品', '印刷和媒介', '文体用品', '石油炼焦和核燃料', '化学原料和化学制品', '医药制造', '化纤制造',
       '橡胶塑料', '非金属', '黑色金属', '有色金属', '金属制品', '通用设备', '专用设备', '汽车制造', '运输设备制造',
       '电气机械和器材制造', '计算机通信和电子设备', '仪器仪表', '其他制造业', '农业', '林业', '畜牧', '渔业',
       '农林牧渔服务', '煤炭开采和洗选', '石油天然气开采', '黑色金属矿采选', '有色金属矿采选', '开采辅助活动',
       '电热生产供应', '燃气生产供应', '水生产供应', '房屋建筑', '土木工程建筑', '建筑安装', '建筑装饰', '批发',
       '零售', '铁路运输', '道路运输', '水上运输', '航空运输', '管道运输', '装卸搬运和运输代理', '仓储', '住宿',
       '餐饮', '电信广电卫星', '互联网服务', '软件信息技术服务', '货币金融服务', '资本市场服务', '保险', '其他金融',
       '租赁', '商务服务', '研究试验发展', '专业技术服务', '科技推广应用服务', '生态环保和治理', '公共设施管理', '卫生',
       '社会工作', '新闻出版', '广电影视录音制作', '文化艺术', '废弃资源综合利用', '邮政', '水利管理', '居民服务',
       '机动车和电子日用品修理', '其他服务', '体育', '娱乐', '非金属矿采选', '其他采矿', '烟草制品',
       '金属制品和设备修理'],
      dtype='object')

In [188]:
citic_ind = stock_data('细分行业月度收盘价.xlsx', '中信一级')
citic_ind.columns

Index(['石油石化', '综合', '传媒', '计算机', '通信', '电子', '房地产', '非银行金融', '银行', '农林牧渔',
       '食品饮料', '医药', '纺织服装', '商贸零售', '汽车', '国防军工', '电力设备及新能源', '机械', '轻工制造',
       '建材', '建筑', '基础化工', '钢铁', '电力及公用事业', '有色金属', '煤炭'],
      dtype='object')

# 进行分析
>ppi = pd.read_excel('CEIC宏观数据.xlsx', sheet_name='PPI', header=0, parse_dates=True, engine='openpyxl')  
fig = px.line(ppi, x="时间", y=["工业生产者出厂价格指数", "生产资料"])  
fig.show()

In [193]:
def compare_plot(ppi, income, stock, inc_stk, price1, price2, title=''):
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_trace(go.Scatter(x=ppi.index, y=ppi, showlegend=True, name='PPI'), secondary_y=False)
    fig.add_trace(go.Scatter(x=income.index, y=income, showlegend=True, name='主营收入'), secondary_y=True)
    fig.add_trace(go.Scatter(x=stock.index, y=stock, showlegend=True, name='产成品库存'), secondary_y=True)
    fig.add_trace(go.Scatter(x=inc_stk.index, y=inc_stk, showlegend=True, name='收入同比-库存同比',
                             line=dict(dash='dash')), secondary_y=True)
    fig.add_trace(go.Scatter(x=price1.index, y=price1, showlegend=True, name='证监会二级指数'), secondary_y=True)
    fig.add_trace(go.Scatter(x=price2.index, y=price2, showlegend=True, name='中信一级指数'), secondary_y=True)

    fig.update_xaxes(title_text=title)
    fig.update_yaxes(title_text="PPI", secondary_y=False)
    fig.update_yaxes(title_text="%", secondary_y=True)

    fig.show()

In [195]:
compare_plot(ppi['煤炭开采和洗选业'], income['煤炭开采和洗选业'], stock['煤炭开采和洗选业'], inc_stk['煤炭开采和洗选业'],
             statistic_ind['煤炭开采和洗选'], citic_ind['煤炭'], title='煤炭产业')